In [1]:
import yaml
import ipywidgets as widgets
from IPython.display import display

class YAMLTagEditor:
    def __init__(self, yaml_file_path):
        self.yaml_file_path = yaml_file_path
        self.load_data()
        self.filter_records()
        self.current_index = 0
        self.create_interface()
        
    def load_data(self):
        with open(self.yaml_file_path, 'r', encoding='utf-8') as file:
            self.data = yaml.safe_load(file)
            
    def save_data(self):
        with open(self.yaml_file_path, 'w', encoding='utf-8') as file:
            yaml.dump(self.data, file, sort_keys=False, allow_unicode=True)
            
    def filter_records(self):
        self.filtered_indices = []
        for i, resource in enumerate(self.data['resources']):
            tags = resource.get('tags', [])
            has_include = any('include in DALIA' in str(tag) for tag in tags)
            has_exclude = any('exclude from DALIA' in str(tag) for tag in tags)
            if not has_include and not has_exclude:
                self.filtered_indices.append(i)
                
    def create_interface(self):
        # Text area to display current record
        self.text_area = widgets.Textarea(
            value='',
            layout=widgets.Layout(width='100%', height='300px'),
            description='Record:',
            disabled=True
        )
        
        # Buttons
        self.include_btn = widgets.Button(
            description='Include in DALIA',
            button_style='success',
            layout=widgets.Layout(width='48%')
        )
        self.exclude_btn = widgets.Button(
            description='Exclude from DALIA',
            button_style='danger',
            layout=widgets.Layout(width='48%')
        )
        
        # Navigation info
        self.nav_label = widgets.Label(value='')
        
        # Event handlers
        self.include_btn.on_click(self.on_include_click)
        self.exclude_btn.on_click(self.on_exclude_click)
        
        # Layout
        button_box = widgets.HBox([self.include_btn, self.exclude_btn])
        self.interface = widgets.VBox([
            self.nav_label,
            self.text_area,
            button_box
        ])
        
        self.update_display()
        
    def update_display(self):
        if not self.filtered_indices:
            self.text_area.value = "No more records to process!"
            self.nav_label.value = "All records processed!"
            self.include_btn.disabled = True
            self.exclude_btn.disabled = True
            return
            
        if self.current_index >= len(self.filtered_indices):
            self.filter_records()  # Refresh filtered records
            if not self.filtered_indices:
                self.text_area.value = "No more records to process!"
                self.nav_label.value = "All records processed!"
                self.include_btn.disabled = True
                self.exclude_btn.disabled = True
                return
            self.current_index = 0
            
        record_idx = self.filtered_indices[self.current_index]
        record = self.data['resources'][record_idx]
        
        # Format record for display
        record_text = yaml.dump(record, sort_keys=False, allow_unicode=True)
        self.text_area.value = record_text
        
        self.nav_label.value = f"Record {self.current_index + 1} of {len(self.filtered_indices)} (Index: {record_idx})"
        
    def on_include_click(self, button):
        self.add_tag("include in DALIA")
        
    def on_exclude_click(self, button):
        self.add_tag("exclude from DALIA")
        
    def add_tag(self, tag):
        if not self.filtered_indices:
            return
            
        record_idx = self.filtered_indices[self.current_index]
        record = self.data['resources'][record_idx]
        
        if 'tags' not in record:
            record['tags'] = []
            
        record['tags'].append(tag)
        
        # Save the file
        self.save_data()
        
        # Move to next record
        self.current_index += 1
        self.update_display()
        
    def display(self):
        display(self.interface)

yaml_file_path = '../resources/nfdi4bioimage.yml'
editor = YAMLTagEditor(yaml_file_path)
editor.display()

## Guidelines
We should include records to DALIA if they:
* Are Bioimaging data related (including data acquisition, analysis, management, visualization) or
* Data literacy related (programming, conceptual)

Acceptable formats:
* Repositories with educational code and notebookds
* Educational posters
* Educational presentations
* Educational Blog posts

Formal requirements:
* Title should be descriptive ("Workshop 2025" is not enough)
* Should not be a page listing more training materials from multiple sources (search engines)
* Links should be persistent (no google docs, dropbox or nextcloud links)